## Q1 Write a python code for North-West corner method.
## Ex. Determine the IBFS of the given T.P. by North-West cost rule
	

In [1]:
import numpy as np

def north_west_corner(supply, demand, cost):
    supply, demand = supply[:], demand[:]   
    m, n = len(supply), len(demand)
    alloc = np.zeros((m, n), dtype=int)
    i = j = 0

    while i < m and j < n:
        qty = min(supply[i], demand[j])
        alloc[i, j] = qty
        supply[i] -= qty
        demand[j] -= qty
        i += supply[i] == 0
        j += demand[j] == 0

    return alloc

# Example
supply = [7, 9, 18]
demand = [5, 8, 7, 14]
cost = [[19, 30, 50, 10],
        [70, 30, 40, 60],
        [40,  8, 70, 20]]

alloc = north_west_corner(supply, demand, cost)
print("Allocation Matrix:\n", alloc)
print("Total Transportation Cost:", np.sum(np.array(cost) * alloc))


Allocation Matrix:
 [[ 5  2  0  0]
 [ 0  6  3  0]
 [ 0  0  4 14]]
Total Transportation Cost: 1015


## Q2. Write a python code for Least Cost Method.Ex. Obtain the IBFS of the following T.P. by Least Cost Method.


In [2]:
import numpy as np

def least_cost_method(supply, demand, cost_matrix):
    supply, demand = np.array(supply), np.array(demand)
    cost_matrix = np.array(cost_matrix, dtype=float)  # make it float
    m, n = len(supply), len(demand)
    allocation = np.zeros((m, n), dtype=int)

    while supply.sum() > 0 and demand.sum() > 0:
        i, j = divmod(np.argmin(cost_matrix), n)  # find least cost cell
        alloc = min(supply[i], demand[j])
        allocation[i, j] = alloc
        supply[i] -= alloc
        demand[j] -= alloc
        if supply[i] == 0: cost_matrix[i, :] = np.inf
        if demand[j] == 0: cost_matrix[:, j] = np.inf

    return allocation

# Example usage
supply = [7, 9, 18]
demand = [5, 8, 7, 14]
cost_matrix = [
    [19, 30, 50, 10],
    [70, 30, 40, 60],
    [40,  8, 70, 20]
]

alloc = least_cost_method(supply, demand, cost_matrix)
print("Allocation Matrix (Least Cost Method):\n", alloc)
print("Total Transportation Cost:", np.sum(np.array(cost_matrix) * alloc))


Allocation Matrix (Least Cost Method):
 [[0 0 0 7]
 [2 0 7 0]
 [3 8 0 7]]
Total Transportation Cost: 814


## Q3. Write a python code for Vogel’s Approximation method.Ex. Obtain the IBFS by VAM


In [ ]:
import numpy as np

def vogels_approximation(cost, supply, demand):
    cost = np.array(cost, dtype=float)
    supply = supply[:]
    demand = demand[:]
    
    m, n = cost.shape
    allocation = np.zeros((m, n))
    
    while sum(supply) > 0 and sum(demand) > 0:
        # Step 1: Row penalties
        row_penalty = []
        for i in range(m):
            if supply[i] > 0:
                valid = [cost[i][j] for j in range(n) if demand[j] > 0]
                if len(valid) >= 2:
                    sorted_row = sorted(valid)
                    row_penalty.append(sorted_row[1] - sorted_row[0])
                elif len(valid) == 1:
                    row_penalty.append(valid[0])
                else:
                    row_penalty.append(-1)
            else:
                row_penalty.append(-1)

        # Step 2: Column penalties
        col_penalty = []
        for j in range(n):
            if demand[j] > 0:
                valid = [cost[i][j] for i in range(m) if supply[i] > 0]
                if len(valid) >= 2:
                    sorted_col = sorted(valid)
                    col_penalty.append(sorted_col[1] - sorted_col[0])
                elif len(valid) == 1:
                    col_penalty.append(valid[0])
                else:
                    col_penalty.append(-1)
            else:
                col_penalty.append(-1)

        # Step 3: Find max penalty
        max_row_pen = max(row_penalty)
        max_col_pen = max(col_penalty)

        if max_row_pen >= max_col_pen:
            row_index = row_penalty.index(max_row_pen)
            valid_cols = [j for j in range(n) if demand[j] > 0]
            col_index = min(valid_cols, key=lambda j: cost[row_index][j])
        else:
            col_index = col_penalty.index(max_col_pen)
            valid_rows = [i for i in range(m) if supply[i] > 0]
            row_index = min(valid_rows, key=lambda i: cost[i][col_index])

        # Step 4: Allocate
        quantity = min(supply[row_index], demand[col_index])
        allocation[row_index][col_index] = quantity
        supply[row_index] -= quantity
        demand[col_index] -= quantity

    return allocation


# Example 

cost = [
    [21, 16, 15, 13],  # S1
    [17, 18, 14, 23],  # S2
    [32, 27, 18, 41]   # S3
]
supply = [11, 13, 19]
demand = [6, 10, 12, 15]

allocation = vogels_approximation(cost, supply, demand)

print("Allocation Matrix (IBFS using VAM):")
print(allocation)

# Total transportation cost
total_cost = (np.array(cost) * allocation).sum()
print("\nTotal Transportation Cost:", total_cost)


Allocation Matrix (IBFS using VAM):
[[ 0.  0.  0. 11.]
 [ 6.  3.  0.  4.]
 [ 0.  7. 12.  0.]]

Total Transportation Cost: 796.0


## Q4. Write a python code for Algebraic Method for solving 2*2 game.

In [4]:
import numpy as np
from itertools import combinations

def solve_lp(c, constraints):
    eqs = [(a, b, rhs) for a, b, _, rhs in constraints]
    pts = [(0,0)]

    # Intersections of constraints
    for (a1,b1,r1),(a2,b2,r2) in combinations(eqs,2):
        try: pts.append(tuple(np.linalg.solve([[a1,b1],[a2,b2]],[r1,r2])))
        except: pass
    # Axis intercepts
    for a,b,rhs in eqs:
        if b: pts.append((0,rhs/b))
        if a: pts.append((rhs/a,0))

    # Feasible points
    feas = []
    for x,y in pts:
        if x>=0 and y>=0 and all(
            (a*x+b*y<=rhs if s=="<=" else a*x+b*y>=rhs)
            for a,b,s,rhs in constraints):
            feas.append((round(x,4),round(y,4)))
    feas = list(set(feas))

    results = [(x,y,c[0]*x+c[1]*y) for x,y in feas]
    return results, max(results,key=lambda t:t[2])

# Example
c = [3,2]
constraints = [(2,1,"<=",18),(2,3,"<=",42),(3,1,"<=",24)]

results, optimal = solve_lp(c,constraints)
print("Feasible Points and Z values:")
for x,y,Z in results: print(f"x={x}, y={y}, Z={Z}")
print("\nOptimal Solution:\n", f"x={optimal[0]}, y={optimal[1]}, Z={optimal[2]}")


Feasible Points and Z values:
x=0, y=0, Z=0
x=6.0, y=6.0, Z=30.0
x=8.0, y=0, Z=24.0
x=3.0, y=12.0, Z=33.0
x=0, y=14.0, Z=28.0

Optimal Solution:
 x=3.0, y=12.0, Z=33.0


## Q5. Write a python program for implementation of simplex algorithm.	

In [5]:
import numpy as np

def simplex(c, A, b):
    A, b, c = np.array(A,float), np.array(b,float), np.array(c,float)
    m, n = A.shape
    T = np.block([[A, np.eye(m), b[:,None]], [-c, np.zeros(m+1)]])  # tableau

    while (T[-1,:-1]<0).any():
        col = np.argmin(T[-1,:-1])
        ratios = [T[i,-1]/T[i,col] if T[i,col]>0 else np.inf for i in range(m)]
        row = np.argmin(ratios)
        if ratios[row]==np.inf: raise Exception("Unbounded solution")
        T[row] /= T[row,col]
        for i in range(m+1):
            if i!=row: T[i] -= T[i,col]*T[row]

    sol = np.zeros(n)
    for i in range(n):
        col = T[:m,i]
        if (col==1).sum()==1 and (col==0).sum()==m-1:
            sol[i] = T[np.where(col==1)[0][0],-1]
    return sol, T[-1,-1], T

# Example
c = [3,2]
A = [[2,1],[2,3],[3,1]]
b = [18,42,24]

sol, val, finalT = simplex(c,A,b)
print("Optimal Solution:", sol)
print("Maximum Z:", val)
print("\nFinal Tableau:\n", finalT)


Optimal Solution: [ 3. 12.]
Maximum Z: 33.0

Final Tableau:
 [[ 0.    1.   -0.5   0.5   0.   12.  ]
 [ 0.    0.   -1.75  0.25  1.    3.  ]
 [ 1.    0.    0.75 -0.25  0.    3.  ]
 [ 0.    0.    1.25  0.25  0.   33.  ]]


## Q6. Write a program to solve the linear programming in pulp problem in Python.

In [1]:
import pulp

# Define LP Problem
prob = pulp.LpProblem("Simple_LP", pulp.LpMaximize)
x1, x2 = pulp.LpVariable('x1', lowBound=0), pulp.LpVariable('x2', lowBound=0)

# Objective
prob += 3*x1 + 2*x2

# Constraints
prob += 2*x1 + x2 <= 18
prob += 2*x1 + 3*x2 <= 42
prob += 3*x1 + x2 <= 24

# Solve
prob.solve()
print("Status:", pulp.LpStatus[prob.status])
print(f"x1={pulp.value(x1)}, x2={pulp.value(x2)}")
print("Max Z =", pulp.value(prob.objective))


Status: Optimal
x1=3.0, x2=12.0
Max Z = 33.0


## Q7. Write a python code for game without saddle point.

In [8]:
def solve_2x2_game(a11, a12, a21, a22):
    d = a11 + a22 - a12 - a21
    if d == 0:
        return print("Saddle point or degenerate game.")
    p1, p2 = (a22 - a21)/d, (a11 - a12)/d
    q1, q2 = (a22 - a12)/d, (a11 - a21)/d
    V = (a11*a22 - a12*a21)/d
    print(f"Player A: p1={p1:.3f}, p2={p2:.3f}")
    print(f"Player B: q1={q1:.3f}, q2={q2:.3f}")
    print(f"Value of Game: V={V:.3f}")

# Example
solve_2x2_game(2, 4, 3, 1)

Player A: p1=0.500, p2=0.500
Player B: q1=0.750, q2=0.250
Value of Game: V=2.500


## Q8. Write a python code for Two person zero sum game with saddle point.

In [12]:
import numpy as np

# Payoff matrix (Player A)
payoff = np.array([[3, 2],
                   [1, 4]])

print("Payoff Matrix:\n", payoff)

# Step 1: Row minima and maximin
row_minima = payoff.min(axis=1)
maximin = row_minima.max()

# Step 2: Column maxima and minimax
col_maxima = payoff.max(axis=0)
minimax = col_maxima.min()

print("Row minima:", row_minima, " -> Maximin:", maximin)
print("Column maxima:", col_maxima, " -> Minimax:", minimax)

# Step 3: Check for saddle point
if maximin == minimax:
    print("\n Saddle point exists with value:", maximin)
    for i in range(payoff.shape[0]):
        for j in range(payoff.shape[1]):
            if payoff[i, j] == maximin:
                print(f"Saddle point at position: Row {i}, Column {j}")
else:
    print("\n No saddle point exists.")


Payoff Matrix:
 [[3 2]
 [1 4]]
Row minima: [2 1]  -> Maximin: 2
Column maxima: [3 4]  -> Minimax: 3

 No saddle point exists.
